# D3S Case Study

Author: Mohamed MESSALTI

Code is partly based on the following 2 articles:

- [Supplier Name Standardization using Unsupervised Learning, Rahul Issar, Oct 6, 2020](https://medium.com/analytics-vidhya/supplier-name-standardization-using-unsupervised-learning-adb27bed9e0d)
- [Company Name Standardization using a Fuzzy NLP Approach, Shashank Gupta, et al. March 28, 2020](https://www.analyticsinsight.net/company-names-standardization-using-a-fuzzy-nlp-approach/)


In [1]:
# pip install nltk
# pip install fuzzywuzzy
# !pip install python-Levenshtein

## Importing the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import time
from collections import Counter

import unicodedata
import re # Regular expression operations

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import nltk # Natural Language Toolkit
nltk.download('wordnet')
nltk.download('omw-1.4')

from fuzzywuzzy import fuzz # to compute the Levenshtein Distance

from sklearn.cluster import AffinityPropagation

import difflib # For the SequenceMatcher to standardize company name of each cluster


[nltk_data] Downloading package wordnet to C:\Users\Mohamed
[nltk_data]     MESSALTI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Mohamed
[nltk_data]     MESSALTI\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Preprocessing
We use standard data pre-processing techniques to clean textual data and remove unwanted words.

In [3]:
# List of keywords to help identify stop_words
company_stopwords=['biz', 'bv', 'co', 'comp', 'company', 
                    'corp','corporation', 'dba', 
                    'inc', 'incorp', 'incorporat', 
                    'incorporate', 'incorporated', 'incorporation', 
                    'international', 'intl', 'intnl', 
                    'limited' ,'llc', 'ltd', 'llp', 
                    'machines', 'pvt', 'pte', 'private', 'unknown',
                    'gmbh', 'sa', 'sarl', 'sas', 'sl', 'sal', 'university',
                    'universite', 'group', 'association', 'agence', 'service']



# Text data encoder function
def filter_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

# Remove special characters & digits (optional) function
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

# Remove company specific stop words
def clean_stopwords(text,eng=False):
    if eng == False:
        custom = company_stopwords
    else:
        custom = company_stopwords + list(ENGLISH_STOP_WORDS)
    for x in custom:
        pattern2 = r'\b'+x+r'\b'
        text=re.sub(pattern2,'',text)
    return text

# Trim the text to remove spaces
def clean_spaces(text):
    text=text.replace('  ', ' ')
    text=text.strip()
    if len(text) < 1:
        text='Tooshorttext'
    return text

# Function to Preprocess Textual data. Provide input as df['Column Name'] to this function
def preprocess_text(column, remove_digits=True, lemm=True, eng=False):
    try:
        column = [filter_ascii(text) for text in column]
        column = [remove_special_characters(text, remove_digits) for text in column]
        column = [text.lower() for text in column] # convert all text to lower case
        column = [clean_stopwords(text, eng) for text in column]
        column = [clean_spaces(text) for text in column]
        ## Lemmatisation (convert the word into root word)
        if lemm == True:
            lem = nltk.stem.wordnet.WordNetLemmatizer()
            column = [lem.lemmatize(text) for text in column]
        return column
    except Exception as e:
        return print(e)

## Clustering

In [4]:
# Function to generate similarity matrix. Provide input as df['Column Name'] to this function
def fuzz_similarity(column):
  similarity_array = np.ones((len(column), (len(column))))*100
  for i in range(1, len(column)):
    for j in range(i):
      s1 = fuzz.token_set_ratio(column.iloc[i],column.iloc[j]) + 0.00000000001
      s2 = fuzz.partial_ratio(column.iloc[i],column.iloc[j]) + 0.00000000001
      similarity_array[i][j] = 2*s1*s2 / (s1+s2)
      
  for i in range(len(column)):
    for j in range(i+1,len(column)):
      similarity_array[i][j] = similarity_array[j][i]
      np.fill_diagonal(similarity_array, 100)
  return similarity_array

In [5]:
# Function to cluster similar name companies. Provide input as dataframe and numpy similarity matrix to this function
def company_clusters(dataframe, matrix):
    dataframe['index'] = dataframe.index 
    cust_ids = dataframe.index.to_list()
    clusters = AffinityPropagation(affinity='precomputed').fit_predict(matrix)
    df_clusters = pd.DataFrame(list(zip(cust_ids, clusters)), columns=['index','Cluster'])
    new = dataframe.merge(df_clusters, 'inner', 'index')
    return new

In [14]:
# Function to map the standard name of each cluster and calculat the confidence of each row with respect to its cluster.
# From the list of substrings for a cluster, we take the one with the highest occurrence (mode), which is considered as the Standard name to be assigned to the current cluster
# Confidence score quantifies the confidence with which we can say that the standard name we identified truly represents the company name for the raw string.
def standard_name(df_eval):
    d_standard_name = {}
    for cluster in df_eval.Cluster.unique():
        names = df_eval[df_eval['Cluster']==cluster].Preprocessed_Text.to_list()
        l_commun_substring = []
        if len(names)>1:
            for i in range(0,len(names)):
                for j in range(i+1,len(names)):
                    SeqMatch = difflib.SequenceMatcher(None, names[i],names[j])
                    match = SeqMatch.find_longest_match(0, len(names[i]), 0, len(names[j]))
                    if (match.size!=0):
                        l_commun_substring.append(names[i][match.a: match.a + match.size].strip())
            n = len(l_commun_substring)
            counts = Counter(l_commun_substring)
            get_mode = dict(counts)
            mode = [k for k, v in get_mode.items() if v==max(list(counts.values()))]
            d_standard_name[cluster] = ";".join(mode)
        else:
            d_standard_name[cluster] = names[0]
    df_standard_names = pd.DataFrame(list(d_standard_name.items()), columns=['Cluster', 'Mapped_name'])
    df_eval = df_eval.merge(df_standard_names, on='Cluster', how='left')
    
    df_eval['Confidence'] = df_eval.apply(lambda x: fuzz.token_set_ratio(x['Mapped_name'], x['Preprocessed_Text']),axis=1)
    return df_eval

## Case Study

### Importing data

In [7]:
df = pd.read_excel("Case_study_names_mapping_test.xlsx")
df = df.drop(['Mapped name'], axis=1)

If you want to run a random subsample, please run the cell below.

In [8]:
# np.random.seed(123) # Fixed seed, so that the experiment can be reproduced.
# subsample_size=100 # number of raw
# df = df.loc[np.random.choice(df.index, subsample_size, replace=False)].copy(deep=True)
# print(df)

### Preprocessing data

In [9]:
df['Preprocessed_Text'] = preprocess_text(column= df['Raw name'], remove_digits=False, lemm=True, eng=False)

### Clustering

In [10]:
start_time = time.time()
similarity_matrix = fuzz_similarity(column=df['Preprocessed_Text'])
end_time = time.time()

print("The execution time to compute the similarity matrix for the data set of size ",
      df.shape[0],"  is :", (end_time-start_time) * 10**3, "ms")

# The execution time to compute the similarity matrix for a data set of size  100   is approx. :     347.46742248535156 ms
# The execution time to compute the similarity matrix for a data set of size  1000  is approx. :   41651.59487724304 ms = 41 seconds
# The execution time to compute the similarity matrix for a data set of size  4895  is approx. : 1383496.4966773987 ms = 23 min

The execution time to compute the similarity matrix for the data set of size  4895   is : 1445896.1153030396 ms


In [11]:
results = company_clusters(df, similarity_matrix)

In [12]:
results = standard_name(results)

### Exporting Results

In [13]:
results.to_excel("results.xlsx") 